In [9]:
import numpy as np

np.random.seed(123)

In [69]:
class CRR:
    def __init__(self, S0, d, u, r, T):
        if not (d < 1 + r < u):
            raise ValueError('in CRR model it must be d < 1 + r < u')
        self.S0 = S0
        self.d = d
        self.u = u
        self.r = r
        self.T = T
        self.p = (1 + r - d)/(u - d)
        self.last_trajectory = np.array([])
        self.S_trajectory = np.array([])
    
    def generate_trajectory(self):
        self.last_trajectory = np.array([])
        
        for i in range(self.T):
            if np.random.uniform(0, 1) < self.p:
                w = self.u
            else:
                w = self.d
            self.last_trajectory = np.append(self.last_trajectory, w)
        return np.ndarray.copy(self.last_trajectory)
        
    def get_S_trajectory(self):
        self.S_trajectory = np.cumprod(self.last_trajectory) * self.S0
        return np.ndarray.copy(self.S_trajectory)
    
    def get_rational_price(self, option):
        trajectory_copy = np.ndarray.copy(self.last_trajectory)
        for i in range(self.T - 1, 0, -1):
            tr = np.ndarray.copy(trajectory_copy)[0:i+1]
            pass
            

In [68]:
class Option:
    def __init__(self, K):
        self.K = K
        self.price_trajectory = np.array([])
        self.X = [0.0, 0.0]
    
    def get_final_price(self, S_trajectory):
        pass

In [53]:
class CallOption(Option):
    def __init__(self, K):
        super().__init__(K)
        
    def get_final_price(self, S_trajectory):
        return np.max(S_trajectory[-1] - self.K, 0)

In [54]:
class PutOption(Option):
    def __init__(self, K):
        super().__init__(K)
        
    def get_final_price(self, S_trajectory):
        return np.max(self.K - S_trajectory[-1], 0)

In [57]:
crr = CRR(100, 0.8, 1.3, 0.1, 10)

In [63]:
crr.generate_trajectory()
S_trajectory = crr.get_S_trajectory()

In [64]:
S_trajectory

array([ 80.        , 104.        , 135.2       , 108.16      ,
       140.608     , 182.7904    , 237.62752   , 190.102016  ,
       247.1326208 , 197.70609664])

In [65]:
call = CallOption(90)
call.get_final_price(S_trajectory)

107.70609664000008